# Projeto 1 da Disciplina de Teoria e Aplicação de Grafos

## Autor: Leandro Beloti Kornelius - 211020900

A Teoria dos Grafos é uma das bases mais importantes da Ciência da Computação tendo inúmeras aplicações em problemas de otimização, análise de redes, redes sociais, redes de transporte e muito mais. Com foi visto na disciplina, um grafo é composto por vértices (ou nós) e arestas (ou ligações), representando, respectivamente, os elementos e as conexões entre eles.

No contexto das redes sociais, os grafos permitem compreender e quantificar o comportamento dos usuários e suas conexões, possibilitando a análise de influência, centralidade, comunidades e padrões de interação. Através de métricas como grau, centralidade de intermediação, centralidade de proximidade e densidade da rede, é possível identificar usuários mais influentes, grupos coesos e a estrutura geral da rede.

Assim, no contexto do projeto, oferece uma ferramenta analítica que pode auxiliar na tomada de decisões, detecção de padrões e compreensão do impacto das interações entre os indivíduos da rede.

Dessa forma, este projeto visa analisar estruturas de redes sociais, utilizando métricas de grafos. Sendo possível avaliar as influências dos usuários da rede social e potenciais de usuários. Para a implementação deste projeto foi usado este jupyter notebook, o qual foi dividido em cinco sessões principais:

1. Coleta de Dados
2. Contrução do Grafo
3. Extração das Métricas Relevantes do Grafo
4. Visualização das Medidas Extraídas de Forma Explícita
5. Elaboração de um Relatório de Análise do Grafo

## 1. Coleta de Dados:

Para realizar o projeto, fomos instruídos a usar o dataset presente na url <https://snap.stanford.edu/data/egonets-Facebook.html>. Os dados deste site foram coletados de um formulário através de um aplicativo do facebook. O dataset foi anonimizado substituindo os ids dos usuários do facebook para um novo valor para cada usuário. Foram também anonimizadas algumas informações como afiliações políticas. Portanto, podemos ver se dois indivíduos tem a mesma orientação política, mas não conseguimos determinar qual ela é.

No arquivo facebook_combined.txt presente no diretório Data > facebook_combined.txt > facebook_combined.txt há os 4039 usuários representados como nós do grafo os quais estão conectados por arestas não direcionadas. Ou seja, se em uma linha deste arquivo tiver que 1 está conectado a 2 isso significa que 2 também está conectado a 1.

Nossa missão é elaborar uma função que seleciona 2000 nós aleatoriamente e todas suas respectivas arestas.

Com isso, devemos preservar todas as arestas entre os nós selecionados. Ou seja, iremos extrair um subgrafo induzido com 2000 vértices do nosso grafo inicial.

Nesse sentido, vamos iniciar com uma função auxiliar que lê o arquivo e retorna um dicionário em que a chave é o identificador numérico de um vértice e o valor correspondentes são todos vértices os quais a chave tem uma aresta que os conecta com direcionamento.

In [80]:
def load_graph_data(file_path: str):
    graph_data = {}
    with open(file_path) as f:
        for line in f:
            origen_v_id, destine_v_id = map(int, line.split())
            if origen_v_id in graph_data:
                graph_data[int(origen_v_id)].append(int(destine_v_id))
            else:
                graph_data[int(origen_v_id)] = [int(destine_v_id)]
            if destine_v_id in graph_data:
                graph_data[int(destine_v_id)].append(int(origen_v_id))
            else:
                graph_data[int(destine_v_id)] = [int(origen_v_id)]
    return graph_data

Essa função recebe o caminho do arquivo que terá os dados acerca das arestas do Grafo. Para cada linha no arquivo, é verificado se o vértice de origem está no dicionário. Caso esteja, é necessário acrescentar o vértice de destino às adjacências do vértice de origem. Caso contrário, tivemos a primeira ocorrência do vértice de origem e criamos uma lista com o primeiro vértice adjacente.

Diante disso, agora precisamos selecionar o subgrafo induzido em que selecionaremos 2000 vértices aleatoriamente deste Grafo. Para isso, usaremos a biblioteca numpy em que vamos gerar um array de 1D contendo 2000 ids entre 0 e 4038.

Com este objetivo em mente a seguinte função auxiliar foi elaborada:

In [81]:
import numpy as np

def generate_2000_random_vs(total_nodes=4039, seed=42):
    rng = np.random.default_rng(seed)
    return rng.choice(total_nodes, size=2000, replace=False)

Para obtenção do subgrafo induzido com os 2000 vértices randomicamente selecionados, nós devemos tratar os dados do Grafo original. Assim, devemos:
* Remover ids e vértices adjacentes não selecionados do dicionário
* Remover vértices adjacentes removidos dos vértices que foram selecionados randômicamente

Contudo, caso já exista o subgrafo induzido gerado, não é necessário gerar um novo, podendo apenas usar o existente.

Sob essa ótica, a função abaixo faz esta implementação com a chamadas das funções para conclusão da primeira etapa. Foram inseridos prints para visualização dos resultados, mas foram comentados para não poluir a saída:

In [82]:
from pathlib import Path

sub_graph_file_path = Path("subgraph.txt")

def generate_sub_graph(graph_data, random_vs):
    if not sub_graph_file_path.exists():
        random_vs = set(random_vs)

        # Creates subgraph
        sub_graph = {
            v: [x for x in adj if x in random_vs]
            for v, adj in graph_data.items() if v in random_vs
        }

        # Saves subgraph
        with open(sub_graph_file_path, "x") as f:
            for v_id, adjs in sub_graph.items():
                for adjacent in adjs:
                    f.write(f"{v_id} {adjacent}\n")

        print("✅ Subgrafo induzido gerado e salvo em 'subgraph.txt'.")
    else:
        print("⚠️ Arquivo com subgrafo induzido já existe. Nenhuma ação foi realizada.")

# Testing the functions implemented:
graph_full_data = load_graph_data("./Data/facebook_combined.txt/facebook_combined.txt")
# print(f"Full graph data dictionary: {graph_full_data}")
random_2000_vs = generate_2000_random_vs()
print("Total gerados:", len(random_2000_vs))
print("Únicos:", len(set(random_2000_vs)))
# print(f"Random vs selected: {random_2000_vs}")
generate_sub_graph(graph_full_data, random_2000_vs)

Total gerados: 2000
Únicos: 2000
⚠️ Arquivo com subgrafo induzido já existe. Nenhuma ação foi realizada.


## 2. Construção do Grafo

Para construção da rede foi instruído o uso do pacote NetworkX o qual a documentação pode ser consultada ao lado <https://networkx.org/documentation/stable/install.html>.

Esta biblioteca permite que nós, a partir de um arquivo, façamos a construção de um grafo, garanta sua conectividade, faça limpezas de arestas, consulte métricas e muito mais.

Assim, a função abaixo constrói o grafo a partir dos 2000 vértices aleatórios gerados na "Etapa 1". Para os vértices isolados, sem arestas incidentes, é feito uma adição explicita destes nós.

In [83]:
import networkx as nx

def build_graph(file_path, nodes):
    G = nx.read_edgelist(file_path, nodetype=int, create_using=nx.Graph())
    # Adds isolated nodes explicitely
    G.add_nodes_from(nodes)
    print(f"🪩 Grafo carregado com {G.number_of_nodes()} nós e {G.number_of_edges()} arestas.")
    print(f"🔹 Nós com grau maior do que 0: {len([n for n, d in G.degree() if d != 0])}")
    print(f"🔹 Nós isolados (sem arestas): {len([n for n, d in G.degree() if d == 0])}")
    return G

# Defining the graph from the random 2000 sample
G = build_graph("subgraph.txt", random_2000_vs)

🪩 Grafo carregado com 2000 nós e 20971 arestas.
🔹 Nós com grau maior do que 0: 1938
🔹 Nós isolados (sem arestas): 62
